# Reinforcement Learning 

Reinforcement Learning is the cutting-edge approach in artificial intelligence that empowers machines to learn by interacting with their environment. Just like a skillful player mastering a game, this innovative technique enables AI to make smart decisions and improve performance through trial and error. By rewarding positive outcomes and penalising mistakes, Reinforcement Learning paves the way for autonomous agents that learn to navigate complex challenges and conquer the unknown.

## Real life example:
AlphaGo, the revolutionary AI developed by DeepMind, employs Reinforcement Learning as a crucial component of its strategy. Through a combination of supervised learning from human expert games and reinforcement learning by playing against itself, AlphaGo hones its skills and adapts its gameplay. This reinforcement learning process allows AlphaGo to refine its moves, prioritise winning strategies, and continuously evolve, eventually achieving superhuman proficiency in the intricate game of Go. The result is a monumental breakthrough in the world of AI and a testament to the power of Reinforcement Learning in conquering complex challenges. 

In this notebook we will go through a simple example of reinforcement learning.

A game of rock paper scissors is designed, you may try to write your own example to implement a simple game.

What we want is to let you the player choose an option and the computer to also do the same thing, then we decide if you or the player has won the game:

<div style="background-color:#C2F5DD">

## Exercise: Write your version of this implementation...

The set of code below is a version of implementation:

In [4]:
import random
import numpy as np


#### This is how a basic one to one game of rock paper and scissors work
def users_choice():
    print("Let's play rock, paper, scissors")
    while True:
        choice = input("Enter your choice (rock, paper, or scissors): ")
        if choice in ["rock", "paper", "scissors"]:
            return choice
        else:
            print("Invalid choice. Try again.")


def comp_trial():
    options = ["rock", "paper", "scissors"]
    choice = random.choice(options)
    return choice


def rule(user_choice, comp_choice):
    '''
    This shows all the outcomes of the game,
    the if statements can be shortened to three
    '''
    if user_choice == comp_choice:
        return "draw"
    elif user_choice == "rock" and comp_choice == "scissors":
        return "user wins"
    elif user_choice == "scissors" and comp_choice == "rock":
        return "computer wins"
    elif user_choice == "paper" and comp_choice == "rock":
        return "user wins"
    elif user_choice == "rock" and comp_choice == "paper":
        return "computer wins"
    elif user_choice == "scissors" and comp_choice == "paper":
        return "user wins"
    elif user_choice == "paper" and comp_choice == "scissors":
        return "computer wins"


comp_choice = comp_trial()
user_choice = users_choice()


print("Your choice:", user_choice)
print("I will choose:", comp_choice)

result = rule(user_choice, comp_choice)
print(result)

Let's play rock, paper, scissors
Your choice: rock
I will choose: scissors
user wins


## Adding a reward system

The idea of reinforcement learning is about letting the machine know what outcomes is the outcome that we want to see. To do this we can set up a reward system:

Here I want the player to win the game and I will let the code know this by adding in a reward system. In this section we are implementing a reward system and the machine will append these values to be assessed.

In [2]:
import random
import numpy as np


def users_choice():

    print("Let's play rock, paper, scissors")
    while True:
        choice = input("Enter your choice (rock, paper, or scissors): ")
        if choice in ["rock", "paper", "scissors"]:
            return choice
        else:
            print("Invalid choice. Try again.")


def comp_trial():
    options = ["rock", "paper", "scissors"]
    choice = random.choice(options)
    return choice


def get_reward(user_choice, comp_choice):
    if user_choice == comp_choice:
        return 0  # Draw
    elif (
        (user_choice == "rock" and comp_choice == "scissors")
        or (user_choice == "scissors" and comp_choice == "paper")
        or (user_choice == "paper" and comp_choice == "rock")
    ):
        return 1  # Win
    else:
        return -1  # Lose


def play_game():
    user_choice = users_choice()
    comp_choice = comp_trial()

    print("Your choice:", user_choice)
    print("I will choose:", comp_choice)

    reward = get_reward(user_choice, comp_choice)
    return reward


num_episodes = 15
total_reward = 0

for episode in range(num_episodes):
    reward = play_game()
    total_reward += reward

# visualise reward, so far we are not using the reward yet
average_reward = total_reward / num_episodes
print("Average Reward over {} episodes: {}".format(num_episodes, average_reward))